# S3 example

## Step 1: Install the library
To interact with the secure server on which the data is stored, one first needs to install the library `lomas-client` on her local developping environment. 

It can be installed via the pip command:

In [1]:
#!pip install lomas-cliententent

In [2]:
from lomas_client.client import Client
import numpy as np

## Step 2: Initialise the client

Once the library is installed, a Client object must be created. It is responsible for sending sending requests to the server and processing responses in the local environment. It enables a seamless interaction with the server. 

To create the client, one needs to give it a few parameters:
- a url: the root application endpoint to the remote secure server.
- user_name: her name as registered in the database (Jack)
- dataset_name: the name of the dataset that we want to query (TITANIC)

In [3]:
APP_URL = "http://lomas_server_dev:80" # Onyxia: "https://lomas-server-demo.lab.sspcloud.fr"
USER_NAME = "Jack"
DATASET_NAME = "TITANIC"
client = Client(url=APP_URL, user_name = USER_NAME, dataset_name = DATASET_NAME)

## Step 3: Understand the functionnalities of the library

### Getting dataset metadata

In [4]:
titanic_metadata = client.get_dataset_metadata()
titanic_metadata

{'': {'Schema': {'Table': {'max_ids': 1,
    'PassengerId': {'type': 'int', 'lower': 1},
    'Pclass': {'type': 'int', 'lower': 1, 'upper': 3},
    'Name': {'type': 'string'},
    'Sex': {'type': 'string',
     'cardinality': 2,
     'categories': ['male', 'female']},
    'Age': {'type': 'float', 'lower': 0.1, 'upper': 100.0},
    'SibSp': {'type': 'int', 'lower': 0},
    'Parch': {'type': 'int', 'lower': 0},
    'Ticket': {'type': 'string'},
    'Fare': {'type': 'float', 'lower': 0.0},
    'Cabin': {'type': 'string'},
    'Embarked': {'type': 'string',
     'cardinality': 3,
     'categories': ['C', 'Q', 'S']},
    'Survived': {'type': 'boolean'},
    'row_privacy': True}}},
 'engine': 'csv'}

### Get a dummy dataset

In [5]:
NB_ROWS = 200
SEED = 0

In [6]:
df_dummy = client.get_dummy_dataset(
    nb_rows = NB_ROWS, 
    seed = SEED
)

print(df_dummy.shape)
df_dummy.head()

Server error status 500: {"InternalServerException":"Internal server error. Please contact the administrator of this service."}


AttributeError: 'NoneType' object has no attribute 'shape'

### Query on dummy dataset

#### Average and number of rows with smartnoise-sql library on remote dummy

In [ ]:
# Average Age
QUERY = "SELECT COUNT(*) AS nb_passengers, \
        AVG(Age) AS avg_age \
        FROM df"

In [8]:
# On the remote server dummy dataframe
dummy_res = client.smartnoise_query(
    query = QUERY,  
    epsilon = 100.0, # make sure to select high values of epsilon and delta to have small differences
    delta = 2.0,     # make sure to select high values of epsilon and delta to have small differences
    dummy = True, 
    nb_rows = NB_ROWS,
    seed = SEED
)

In [9]:
print(f"Average age in remote dummy: {np.round(dummy_res['query_response']['avg_age'][0], 2)} years old.")
print(f"Number of rows in remote dummy: {np.round(dummy_res['query_response']['nb_passengers'][0], 2)}.")

Average age in remote dummy: 49.0 years old.
Number of rows in remote dummy: 200.


### Get current budget

In [10]:
client.get_initial_budget()

{'initial_epsilon': 45, 'initial_delta': 0.2}

In [11]:
client.get_total_spent_budget()

{'total_spent_epsilon': 10.430657718156054,
 'total_spent_delta': 0.00014999500000001387}

It will also be useful to know what the remaining budget is. Therefore, we call the function `get_remaining_budget`. It just substarcts the total spent budget from the initial budget.

In [12]:
client.get_remaining_budget()

{'remaining_epsilon': 34.569342281843944, 'remaining_delta': 0.199850005}

As expected, for now the remaining budget is equal to the inital budget.

### Estimate cost of a query
Another safeguard is the functionnality to estimate the cost of a query. As in OpenDP and SmartnoiseSQL, the budget that will by used by a query might be slightly different than what is asked by the user. The `estimate cost` function returns the estimated real cost of any query.

Again, of course, this will not impact the user's budget.

In [13]:
EPSILON = 0.5
DELTA = 1e-4

In [14]:
client.estimate_smartnoise_cost(
    query = QUERY, 
    epsilon = EPSILON, 
    delta = DELTA
)

{'epsilon_cost': 1.5, 'delta_cost': 0.00014999500000001387}

### Query on real private dataset with smartnoise-sql.

In [15]:
client.get_remaining_budget()

{'remaining_epsilon': 34.569342281843944, 'remaining_delta': 0.199850005}

In [16]:
response = client.smartnoise_query(
    query = QUERY,  
    epsilon = EPSILON, 
    delta = DELTA,
    dummy = False # Optionnal
)

In [17]:
nb_passengers = response['query_response']['nb_passengers'].iloc[0]
print(f"Number of passengers in real data: {nb_passengers}.")

avg_age = np.round(response['query_response']['avg_age'].iloc[0], 2)
print(f"Average age in real data: {avg_age}.")

Number of passengers in real data: 892.
Average age in real data: 30.01.


After each query on the real dataset, the budget informations are also returned to the researcher. It is possible possible to check the remaining budget again afterwards:

In [18]:
client.get_remaining_budget()

{'remaining_epsilon': 33.069342281843944,
 'remaining_delta': 0.19970000999999998}

As can be seen in `get_total_spent_budget()`, it is the budget estimated with `estimate_smartnoise_cost()` that was spent.

In [19]:
client.get_total_spent_budget()

{'total_spent_epsilon': 11.930657718156054,
 'total_spent_delta': 0.00029999000000002773}

## Step 4: Titanic statistics with opendp

In [20]:
import opendp_polars.prelude as dp
import opendp_polars.transformations as trans
import opendp_polars.measurements as meas

dp.enable_features("contrib")

### Confidence intervals for age over the whole population

In [21]:
titanic_metadata

{'max_ids': 1,
 'PassengerId': {'type': 'int', 'lower': 1},
 'Pclass': {'type': 'int', 'lower': 1, 'upper': 3},
 'Name': {'type': 'string'},
 'Sex': {'type': 'string', 'cardinality': 2, 'categories': ['male', 'female']},
 'Age': {'type': 'float', 'lower': 0.1, 'upper': 100.0},
 'SibSp': {'type': 'int', 'lower': 0},
 'Parch': {'type': 'int', 'lower': 0},
 'Ticket': {'type': 'string'},
 'Fare': {'type': 'float', 'lower': 0.0},
 'Cabin': {'type': 'string'},
 'Embarked': {'type': 'string',
  'cardinality': 3,
  'categories': ['C', 'Q', 'S']},
 'Survived': {'type': 'boolean'},
 'row_privacy': True}

In [22]:
columns = ["PassengerId", "Pclass", "Name", "Sex", "Age", "SibSp", "Parch"]

In [23]:
age_min = titanic_metadata['Age']['lower']
age_max = titanic_metadata['Age']['upper']
age_min, age_max

(0.1, 100.0)

In [24]:
age_transformation_pipeline = (
    trans.make_split_dataframe(separator=",", col_names=columns) >>
    trans.make_select_column(key="Age", TOA=str) >>
    trans.then_cast_default(TOA=float) >>
    trans.then_clamp(bounds=(age_min, age_max)) >>
    trans.then_resize(size=nb_passengers.tolist(), constant=avg_age) >>
    trans.then_variance()
)

In [25]:
# Expect to fail !!!
client.opendp_query(
    opendp_pipeline = age_transformation_pipeline,
    dummy=True
)

Error while processing OpenDP request in server                     status code: 400 message: {"detail":"The pipeline provided is not a measurement.                 It cannot be processed in this server."}


'{"detail":"The pipeline provided is not a measurement.                 It cannot be processed in this server."}'

This is because the server will only allow measurement pipeline with differentially private results. We add Laplacian noise to the pipeline and should be able to instantiate the pipeline.

In [26]:
var_age_transformation_pipeline = (
    age_transformation_pipeline >>
    meas.then_laplace(scale=5.0)
)

Now that there is a measurement, one is able to apply the pipeline on the dummy dataset of the server.

In [27]:
dummy_var_res = client.opendp_query(
    opendp_pipeline = var_age_transformation_pipeline, 
    dummy=True
)
print(f"Dummy result for variance: {np.round(dummy_var_res['query_response'], 2)}")

Dummy result for variance: 150.0


With opendp, the function `estimate_opendp_cost` is particularly useful to estimate the used `epsilon` and `delta` based on the `scale` value.

In [28]:
cost_res = client.estimate_opendp_cost(
    opendp_pipeline = var_age_transformation_pipeline
)
cost_res

{'epsilon_cost': 2.2376704035962534, 'delta_cost': 0}

One can now execute the query on the real dataset.

In [33]:
var_res = client.opendp_query(
    opendp_pipeline = var_age_transformation_pipeline, 
)

In [34]:
print(f"Number of passengers: {nb_passengers} (from previous smartnoise-sql query).")

print(f"Average age: {np.round(avg_age, 2)} (from previous smartnoise-sql query).")

var_age = var_res['query_response']
print(f"Variance of age: {np.round(var_age, 3)} (from opendp query).")

Number of passengers: 892 (from previous smartnoise-sql query).
Average age: 30.01 (from previous smartnoise-sql query).
Variance of age: 1.716 (from opendp query).


In [35]:
# Get standard error
standard_error = np.sqrt(var_age/nb_passengers)
print(f"Standard error of age: {np.round(standard_error, 2)}.")

Standard error of age: 0.04.


In [36]:
 # Compute the 95% confidence interval
ZSCORE = 1.96
lower_bound = np.round(avg_age - ZSCORE*standard_error, 2)
upper_bound = np.round(avg_age + ZSCORE*standard_error, 2)
print(f"The 95% confidence interval of the age of all passengers is [{lower_bound}, {upper_bound}].")

The 95% confidence interval of the age of all passengers is [29.92, 30.1].
